In [1]:
!pip install transformers inseq datasets

Looking in indexes: https://pypi.org/simple, https://pypi.org/simple


In [2]:
from datasets import load_dataset
dataset = load_dataset("AmazonScience/massive", "en-US", split='test')

/Users/nelenj/miniconda3/envs/phoenix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since AmazonScience/massive couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'en-US' at /Users/nelenj/.cache/huggingface/datasets/AmazonScience___massive/en-US/1.0.0/ff6bd8e4b27c3543e4f8fe2108f32bb95a6f8740 (last modified on Tue Jan 23 14:28:16 2024).


In [22]:
import inseq
name = "qanastek/XLMRoberta-Alexa-Intents-Classification"

model = inseq.load_model("gpt2", "saliency")
attributes = model.attribute(
    "Hello ladies and",
    generation_args={"max_new_tokens": 9},
    n_steps=500,
    internal_batch_size=50
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Unused arguments during attribution: {'n_steps': 500, 'internal_batch_size': 50}
Attributing with saliency...: 100%|██████████| 12/12 [00:00<00:00, 11.80it/s]


In [23]:
attributes.show()

,Ġgentlemen,",",ĠI,Ġam,Ġa,Ġmember,Ġof,Ġthe,ĠBoard
Hello,0.302,0.234,0.171,0.249,0.106,0.199,0.133,0.092,0.156
Ġladies,0.509,0.283,0.227,0.292,0.119,0.243,0.141,0.093,0.151
Ġand,0.189,0.21,0.213,0.085,0.255,0.082,0.09,0.185,0.075
Ġgentlemen,,0.274,0.148,0.189,0.099,0.195,0.109,0.094,0.156
",",,,0.24,0.073,0.223,0.057,0.072,0.154,0.054
ĠI,,,,0.112,0.083,0.061,0.062,0.046,0.054
Ġam,,,,,0.115,0.08,0.053,0.034,0.05
Ġa,,,,,,0.084,0.085,0.131,0.052
Ġmember,,,,,,,0.254,0.09,0.172
Ġof,,,,,,,,0.081,0.029


In [24]:
attributes.show()
attributes.sequence_attributions[0].target_attributions.shape

,Ġgentlemen,",",ĠI,Ġam,Ġa,Ġmember,Ġof,Ġthe,ĠBoard
Hello,0.302,0.234,0.171,0.249,0.106,0.199,0.133,0.092,0.156
Ġladies,0.509,0.283,0.227,0.292,0.119,0.243,0.141,0.093,0.151
Ġand,0.189,0.21,0.213,0.085,0.255,0.082,0.09,0.185,0.075
Ġgentlemen,,0.274,0.148,0.189,0.099,0.195,0.109,0.094,0.156
",",,,0.24,0.073,0.223,0.057,0.072,0.154,0.054
ĠI,,,,0.112,0.083,0.061,0.062,0.046,0.054
Ġam,,,,,0.115,0.08,0.053,0.034,0.05
Ġa,,,,,,0.084,0.085,0.131,0.052
Ġmember,,,,,,,0.254,0.09,0.172
Ġof,,,,,,,,0.081,0.029


torch.Size([12, 9, 768])

In [27]:
attributes.sequence_attributions[0].target_attributions

tensor([[[1.0344e-02, 2.7077e-02, 1.0581e-02,  ..., 5.1415e-02,
          2.6066e-02, 7.5217e-02],
         [4.2698e-03, 4.3943e-03, 3.8190e-03,  ..., 4.6876e-03,
          2.8288e-03, 1.5631e-02],
         [7.5155e-04, 5.1236e-04, 9.2103e-04,  ..., 1.3987e-03,
          6.1990e-04, 4.2403e-03],
         ...,
         [7.8875e-04, 5.2713e-05, 2.5445e-04,  ..., 1.9343e-05,
          1.5100e-04, 2.7782e-04],
         [6.5900e-03, 1.3054e-05, 6.9487e-04,  ..., 3.5940e-04,
          8.0766e-03, 5.0390e-03],
         [2.6404e-03, 2.4977e-04, 4.3185e-04,  ..., 1.0562e-03,
          1.7282e-03, 3.5987e-03]],

        [[6.6942e-02, 2.7756e-02, 2.2101e-03,  ..., 2.0206e-01,
          1.2519e-01, 3.5930e-04],
         [3.8699e-03, 2.1068e-03, 4.4965e-03,  ..., 1.3168e-02,
          9.9988e-03, 4.3497e-03],
         [1.2809e-03, 2.1093e-04, 1.7163e-03,  ..., 1.0267e-02,
          8.3779e-03, 1.9233e-03],
         ...,
         [9.6775e-04, 2.1327e-04, 3.8011e-04,  ..., 4.7221e-04,
          8.939

In [30]:
targets = attributes.sequence_attributions[0]

In [89]:
l = targets.aggregate().target_attributions.detach().numpy()

In [90]:
import numpy
q = numpy.nan_to_num(l)
q

array([[0.30167693, 0.23359624, 0.17100681, 0.24863277, 0.10623145,
        0.19914265, 0.13328516, 0.0917097 , 0.15601338],
       [0.5091447 , 0.2826379 , 0.2268443 , 0.29196757, 0.11906777,
        0.2434777 , 0.14066824, 0.09301255, 0.15149476],
       [0.18917838, 0.20988022, 0.2132668 , 0.08470584, 0.25459957,
        0.0818764 , 0.09034701, 0.1851784 , 0.07463957],
       [0.        , 0.27388567, 0.14846985, 0.18934165, 0.09887534,
        0.19452916, 0.10891982, 0.09395655, 0.1564939 ],
       [0.        , 0.        , 0.24041225, 0.07292711, 0.2227202 ,
        0.05652193, 0.0717023 , 0.15405983, 0.05403432],
       [0.        , 0.        , 0.        , 0.11242503, 0.08330294,
        0.06119951, 0.06225747, 0.04638188, 0.0535332 ],
       [0.        , 0.        , 0.        , 0.        , 0.11520271,
        0.07956768, 0.05323697, 0.03384377, 0.05034051],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.08368495, 0.08525425, 0.13059467, 0.05248461],


In [91]:
import requests

url = "http://localhost:4000"

import numpy
import json


dump = {
    "model": name,
    "text": "Hello ladies and",
    "attributes": q
}

In [92]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (numpy.int_, numpy.intc, numpy.intp, numpy.int8,
            numpy.int16, numpy.int32, numpy.int64, numpy.uint8,
            numpy.uint16,numpy.uint32, numpy.uint64)):
            return int(obj)
        elif isinstance(obj, (numpy.float_, numpy.float16, numpy.float32, 
            numpy.float64)):
            return float(obj)
        elif isinstance(obj,(numpy.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
dump =  json.dumps(dump)

In [93]:
def send_body(body):
    # posting it to the xai-platform
    u = f"{url}/runs"
    requests.post(u, data= body, headers={'Content-Type': 'application/json'})

send_body(dump)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))